In [1]:
import os, os.path, codecs
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import decomposition
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import numpy as np
import pandas as pd
import json


In [2]:
def reading_in(file, listname):
    with open(file) as json_data:
            d = json.load(json_data)
            for i in d:
                listname.append(i)
            json_data.close()


In [3]:
s114 = []
reading_in('114_2.txt', s114)
df = pd.DataFrame(s114)
speeches = df.text
len(speeches)

7186

In [114]:
tfidf = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS, ngram_range=(1,2), lowercase=True, strip_accents="unicode", token_pattern='\\b[a-z][a-z][a-z][a-z][a-z]+\\b', norm="l2",  use_idf=True) 
A = tfidf.fit_transform(speeches)

In [115]:
num_terms = len(tfidf.vocabulary_)
terms = [""] * num_terms
for term in tfidf.vocabulary_.keys():
	terms[ tfidf.vocabulary_[term] ] = term

In [118]:
model = decomposition.NMF(init="nndsvd", n_components=12, max_iter=400)
W = model.fit_transform(A)
H = model.components_



In [169]:

for topic_index in range( H.shape[0] ):
    top_indices = np.argsort( H[topic_index,:] )[::-1][0:50
                                                      ]
    term_ranking = [terms[i] for i in top_indices]
    print ("Topic %d: %s" % ( topic_index, ", ".join( term_ranking ) ))

Topic 0: indian, education, elementary, school, secondary, schools, indian country, elementary secondary, secondary education, rural poverty, areas indian, poverty areas, identifies, poverty, students, section, rural, study, increase school, education rural, education collaboration, school secondary, school graduation, indian tribe, collaboration secretary, graduation, secondary schools, given section, meaning given, elementary school, tribe, areas, secretary education, secretary, shall, interior shall, country, child, given, report, collaboration, definitions, meaning, interior, secretary interior, enactment secretary, amendment, tribes, indian tribes, later enactment
Topic 1: military, defense, selective service, selective, arsenal, military manufacturing, manufacturing arsenal, activities department, exchanges, india, activities, military selective, manufacturing, enhance, fiscal, joint military, department, military operations, department defense, counter, engagement, authorize, co

In [120]:
topics =['Education', 'Military','Economy', 'Nominations', 'Energy', 'Programs and Agencies', 'World Economy', 'Nuclear Agreements', 'Healthcare', 'Amendments', 'Economy', 'Formal Speech']

In [121]:
test = model.transform(A)


In [122]:
len(test)

7186

In [142]:
test2 = []
for i in test:
    j = np.delete(i, 11)
    test2.append(j)

In [143]:
test2[2]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.00102587,  0.        ,  0.        ,  0.0040529 ,  0.        ,  0.        ])

In [144]:
maxtopic = []
for i in test2:
    maxtopic.append(np.argmax(i))


In [145]:
len(maxtopic)

7186

In [146]:
maxtopic[:20]

[2, 3, 8, 9, 9, 9, 9, 9, 9, 5, 3, 3, 8, 3, 8, 7, 3, 3, 3, 8]

In [147]:
topic_list = []
for i in maxtopic:
    topic = topics[i]
    topic_list.append(topic)

In [148]:
topic_list[:20]

['Economy',
 'Nominations',
 'Healthcare',
 'Amendments',
 'Amendments',
 'Amendments',
 'Amendments',
 'Amendments',
 'Amendments',
 'Programs and Agencies',
 'Nominations',
 'Nominations',
 'Healthcare',
 'Nominations',
 'Healthcare',
 'Nuclear Agreements',
 'Nominations',
 'Nominations',
 'Nominations',
 'Healthcare']

In [149]:
df['topic']=topic_list

In [150]:
df['topic'].value_counts()

Amendments               1081
Healthcare               1026
Energy                    892
Education                 751
Nuclear Agreements        694
Economy                   687
Military                  678
Programs and Agencies     624
Nominations               525
World Economy             228
Name: topic, dtype: int64

In [151]:
roster = pd.read_csv('roster114.csv')

In [152]:
names = roster.title

In [153]:
lastnames = []
for i in names:
    splitnames = i.split(' ')
    lastname = splitnames[-1].lower()
    if lastname == 'Jr.':
        lastnames.append(splitnames[-2].lower())
    else:
        
        lastnames.append(lastname)

In [154]:
roster['lastnames'] = lastnames
rosterparties = roster.party
rosterstates = roster.state

In [155]:
party = []
state = []
for i in df.speaker:
    try:
        index = lastnames.index(i.lower())
        party.append(rosterparties[index])
        state.append(rosterstates[index])
    except ValueError:
        party.append('not found')
        state.append('not found')

In [156]:
df['party']=party
df['state'] = state

In [157]:
df

,date,session,speaker,text,url,topic,party,state
0,2015-01-06,114,McCONNELL,"McCONNELL. Mr. President, I send to the desk, ...",http://www.congress.gov/congressional-record/2...,Economy,R,KY
1,2015-01-06,114,DURBIN,"DURBIN. Mr. President, I thank the majority le...",http://www.congress.gov/congressional-record/2...,Nominations,D,IL
2,2015-01-06,114,HATCH,"HATCH. Mr. President, it is with a heavy heart...",http://www.congress.gov/congressional-record/2...,Healthcare,R,UT
3,2015-01-06,114,MURPHY,"MURPHY (for himself, Mr. Blumenthal, Mr. Brown...",http://www.congress.gov/congressional-record/2...,Amendments,D,CT
4,2015-01-06,114,MURPHY,"MURPHY (for himself, Mr. Blumenthal, Mr. Brown...",http://www.congress.gov/congressional-record/2...,Amendments,D,CT
5,2015-01-06,114,MURPHY,"MURPHY (for himself, Mr. Blumenthal, Mr. Brown...",http://www.congress.gov/congressional-record/2...,Amendments,D,CT
6,2015-01-06,114,MURPHY,"MURPHY (for himself, Mr. Blumenthal, Mr. Brown...",http://www.congress.gov/congressional-record/2...,Amendments,D,CT
7,2015-01-06,114,MURPHY,"MURPHY (for himself, Mr. Blumenthal, Mr. Brown...",http://www.congress.gov/congressional-record/2...,Amendments,D,CT
8,2015-01-06,114,McCONNELL,"McCONNELL (for himself, Mr. Reid of Nevada, Ms...",http://www.congress.gov/congressional-record/2...,Amendments,R,KY
9,2015-01-06,114,UDALL,"UDALL (for himself, Mr. Merkley, Mr. Blumentha...",http://www.congress.gov/congressional-record/2...,Programs and Agencies,D,NM


In [161]:
df = df.drop('session' , axis = 1)
df = df.drop('text', axis=1)
df = df.drop('url', axis=1)
df

,date,speaker,topic,party,state
0,2015-01-06,McCONNELL,Economy,R,KY
1,2015-01-06,DURBIN,Nominations,D,IL
2,2015-01-06,HATCH,Healthcare,R,UT
3,2015-01-06,MURPHY,Amendments,D,CT
4,2015-01-06,MURPHY,Amendments,D,CT
5,2015-01-06,MURPHY,Amendments,D,CT
6,2015-01-06,MURPHY,Amendments,D,CT
7,2015-01-06,MURPHY,Amendments,D,CT
8,2015-01-06,McCONNELL,Amendments,R,KY
9,2015-01-06,UDALL,Programs and Agencies,D,NM


In [162]:
fullparty = []
for i in df.party:
    if i == 'R':
        fullparty.append('Republican')
    elif i == 'D':
        fullparty.append('Democrat')
    else:
        fullparty.append('Independent')

In [163]:
len(fullparty)

7186

In [165]:
df=df.drop('party', axis=1)
df['party']=fullparty
df['dummycount']= 1

In [166]:
df

,date,speaker,topic,state,party,dummycount
0,2015-01-06,McCONNELL,Economy,KY,Republican,1
1,2015-01-06,DURBIN,Nominations,IL,Democrat,1
2,2015-01-06,HATCH,Healthcare,UT,Republican,1
3,2015-01-06,MURPHY,Amendments,CT,Democrat,1
4,2015-01-06,MURPHY,Amendments,CT,Democrat,1
5,2015-01-06,MURPHY,Amendments,CT,Democrat,1
6,2015-01-06,MURPHY,Amendments,CT,Democrat,1
7,2015-01-06,MURPHY,Amendments,CT,Democrat,1
8,2015-01-06,McCONNELL,Amendments,KY,Republican,1
9,2015-01-06,UDALL,Programs and Agencies,NM,Democrat,1


In [168]:
df.to_csv('wednesday.csv', index=False)